
# **$\color{orange}{\text{Flower Classification on TPU  }}$**


Kaggle'a TPU seçeneği gelmiş, hoşgelmiş. GPU'dan ziyade TPU'lar çok daha hızlı işlem yapan birimlerdir. Biz de TPU üzerinde temel CNN algoritmalarını deneyerek, TPU kullanımını test etmiş olacağız. Temel fonksiyonlar için Martin Görner'e teşekkür ederim. Öncü ve yönlendirici çalışmasından esinlenerek türkçe bir kaynak oluşturmak istedim. Amacım, ingilizce kaynakları takip edemeyen Türk geliştiriciler için yardımcı bir döküman oluşturmak.

Çalışma içerisinde:
* TPU Mimarisi ve GPU ve CPU'lara göre avantajları
* TPU kullanımı ve hazırlanması
* Çiçek sınıflandırması üzerinde CNN algoritmaları ile pre-trained modeller ile denemeler yapılması

Bulunmaktadır.


**EfficientNetB6, EfficientNetB7 ve DenseNet201 kullanılmıştır. 20 epoch ile tamamlanmıştır.**

****Kısa Özet: 9 epoch'da model performansları****


**$\color{orange}{\text{9 Epoch'da EfficientNetB6 için : }}$**

* sparse_categorical_accuracy: 0.9394 - val_loss: 0.4078 - val_sparse_categorical_accuracy: 0.9108



**$\color{orange}{\text{9 Epoch'da DenseNet201 için : }}$**

* sparse_categorical_accuracy: 0.9458 - val_loss: 0.4519 - val_sparse_categorical_accuracy: 0.8898






# **$\color{pink}{\text{Ön Çalışma ve İşleme  }}$**

* Gerekli Kütüphaneler yüklenmiştir. -> tensorflow ve keras modelleri kullanılmıştır

* Seed çekirdeği atanmıştır. -> Random atanan değerler için aynı kökten (tohumdan) alması sağlanır.

In [ ]:
!pip install --quiet efficientnet
print("VolkanOzdemirDL")
import math, os, re, warnings, random
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, applications, Sequential, losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import efficientnet.tfkeras as efn
from IPython.display import SVG


def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings("ignore")

# $\color{orange}{\text{TPU Nedir? }}$
 


TPU: "Tensor Processimg Unit", Türkçe olarak ise tensor işlem birimi büyük data yığınları ve konvalosyonel sinir ağları ( CNN) için son derece optimize edilmiş çok yüksek eğitim verimine sahip donanımlardır. 

Daha önce data science veya derin öğrenme ile uğraşmışsanız illaki bu algoritmaların GPU'larda CPU'lara göre ne kadar hızlı olduğunu duymuşsunuzdur. Üstelik tek fark da bu değil, CPU'lar kapasite gereği büyük batch-size kullanamamaktadır ve başarımı yükseltmek için gerekli olan giriş büyüklüğüne (input size) ulaşamamaktadır. CPU'nun 1 ila 8 çekirdeği veya daha fazlası vardır. Bir GPU'da ise yüzlerce vardır. Bu işlem birimlerinin fazla olması yapay sinir ağları gibi toplama ve çarpma gibi temel işlemler üzerine oluşturulmuş ( feed-forward yapılar için wX+b ve aktivasyon fonksiyonları ve back-propogation için gradient descent ) sistemler için çok hızlı işlem yapabilmektedir. GPU ve TPU ise çok benzer teknolojidir. Tek fark,  GPU üreticilerinin kimseye satmadıkları tescilli GPU yongalarını kullanan bir bulut hizmeti olmasıdır. Şu an en ünlü TPU paylaşımcısı Google'dır.


CPU, GPU ve TPU'nun farkını anlamak için aslında temel algebra'dan yararlanmak gerekmektedir. CPU işleme birimini skaler bir değer olarak düşünecek olur isek, GPU bir vektör gibi, TPU ise bir matris ya da çok boyutlu matris olan tensor'ler ile ifade edilebilir. Tensor yapıları  çok boyutlu matrisler olarak düşünülmelidir ki CNN'lerin temelinde 3 boyutlu bir RGB resim girişi N boyutlu tensorlere dönmektedir.

Giriş data boyutu: 
* CPU: 1 X 1 data birimi
* GPU: 1 X N data birimi
* TPU: N X N data birimi

Performans olarak:


* CPU döngü başına onlarca işlemi idare edebilir.
* GPU, döngü başına on binlerce işlemi kaldırabilir.
* TPU, döngü başına 128.000'e kadar işlemi gerçekleştirebilir.
 
Tabi ki, TPU'nun kullanımı uygun olarak ayarlamak ve kullanıma hazır hale getirmek gerekmektedir.



Tek bir Cloud TPU cihazı, her biri iki TPU çekirdeğine sahip dört yongadan oluşur. Yine de Cloud TPU'nun verimli kullanımı için, bir algoritma programı sekiz çekirdeğin her birini kullanmalıdır. "TPUEstimator" çoğaltılmış bir hesaplama oluşturmak ve çalıştırmak için bir grafik operatörü sağlar. Her Replica, esasen, her bir çekirdek üzerinde çalıştırılan ve toplam parti boyutunun 1 / 8'ini içeren bir mini parti (mini batch) eğiten, eğitim grafiğinin bir kopyasıdır. Yani eğitimimiz için çalışan ufak minik grafik hesaplayıcılardır. Replica sayısı ile bir çok hyper-parametreyi ayarlayacağız.


# Ön Çalışma ve İşleme 

* TPU Donanım varlığı test edilmiştir.

> tpu = tf.distribute.cluster_resolver.TPUClusterResolver()   # -> TPU kümesi hakkında bilgi sağlar

> tf.config.experimental_connect_to_cluster(tpu)

> tf.tpu.experimental.initialize_tpu_system(tpu)

> strategy = tf.distribute.experimental.TPUStrategy(tpu)  # ->  TPU configi yapılıyor, ön değerleniyor ve strategy tanımlanıyor


> REPLICAS = strategy.num_replicas_in_sync   # -> Gradyan geçişlerinin toplandığı kopya (Replicas) sayısını döndürür.


In [ ]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

Model Parametreleri girilmiştir. TPU'da GPU'da eğitimden farklı olarak strategy tanımlanmakta ve REPLICAS ile parametreler parametrize edilmektedir. Örneğin Batch size 4 * REPLICAS ( bizim durumumuzda 4 * 8=32 olarak) ifade edilmiştir. Burada batch size artırılabilir ve artırılması local minimumlara takılmayı engeller fakat yeterli hafıza olmaması durumunda kodunuz çalışmaz. Eğer kodunuz memory hatası veriyorsa, el mahkum, bu sayıyı küçültmek gerekecektir.

Yarışma konusu sınıfalar da teker teker yazılmıştır. Bu bölgedeki kodlar için Martin Görner, https://www.kaggle.com/mgornergoogle 'in çalışmasından yardım alınmıştır ve kendi algoritmamıza göre düzenlenmiştir.

In [ ]:
BATCH_SIZE = 4 * REPLICAS
WARMUP_EPOCHS = 3
WARMUP_LEARNING_RATE = 1e-4 * REPLICAS
EPOCHS =30
LEARNING_RATE = 3e-5 * REPLICAS
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
N_CLASSES = 104
ES_PATIENCE = 5

model_path = f'model_{HEIGHT}x{WIDTH}.h5'

GCS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started') + f'/tfrecords-jpeg-{HEIGHT}x{WIDTH}'

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

# Bunu tek tek yazmanıza gerek yok ama inputu tf records olarak verdiklerinden ön işleme uzun sürüyor
CLASSES = [
    'pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 
    'wild geranium', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 
    'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 
    'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 
    'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 
    'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 
    'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 
    'carnation', 'garden phlox', 'love in the mist', 'cosmos',  'alpine sea holly', 
    'ruby-lipped cattleya', 'cape flower', 'great masterwort',  'siam tulip', 
    'lenten rose', 'barberton daisy', 'daffodil',  'sword lily', 'poinsettia', 
    'bolero deep blue',  'wallflower', 'marigold', 'buttercup', 'daisy', 
    'common dandelion', 'petunia', 'wild pansy', 'primula',  'sunflower', 
    'lilac hibiscus', 'bishop of llandaff', 'gaura',  'geranium', 'orange dahlia', 
    'pink-yellow dahlia', 'cautleya spicata',  'japanese anemone', 
    'black-eyed susan', 'silverbush', 'californian poppy',  'osteospermum', 
    'spring crocus', 'iris', 'windflower',  'tree poppy', 'gazania', 'azalea', 
    'water lily',  'rose', 'thorn apple', 'morning glory', 'passion flower',  
    'lotus', 'toad lily', 'anthurium', 'frangipani',  'clematis', 'hibiscus', 
    'columbine', 'desert-rose', 'tree mallow', 'magnolia', 'cyclamen ', 
    'watercress',  'canna lily', 'hippeastrum ', 'bee balm', 'pink quill',  
    'foxglove', 'bougainvillea', 'camellia', 'mallow',  'mexican petunia',  
    'bromelia', 'blanket flower', 'trumpet creeper',  'blackberry lily', 
    'common tulip', 'wild rose']

* Decode fonksiyonu
* Etiketli datayı tf record'dan okuma fonksiyonu
* Etiketlenmemiş datayı tf record'dan okuma fonksiyonu
* Data seti yükleme
* Data augmentation --> sağ-sol aynalama, yukarı aşağı aynalama, 90 derece döndürme, rasgele kesme, resize
* Diğer data ve data görselleştirme fonksiyonları

In [ ]:

AUTO = tf.data.experimental.AUTOTUNE # instructs the API to read from multiple files if available.

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [HEIGHT, WIDTH, 3])
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    crop_size = tf.random.uniform([], int(HEIGHT*.75), HEIGHT, dtype=tf.int32)
        
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, lower=0, upper=2)
    image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
    image = tf.image.resize(image, size=[HEIGHT, WIDTH])
    image = tf.image.rot90(image)
    return image, label

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_training_dataset_preview(ordered=True):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:

np.set_printoptions(threshold=15, linewidth=80)

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)

def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()
    
# Visualize model predictions
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower_eval(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=14, color='red' if red else 'black')
    return subplot+1

def display_9_images_with_predictions(images, predictions, labels):
    subplot=331
    plt.figure(figsize=(13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower_eval(image, title, subplot, not correct)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

# DATA

* Data Train - Validation - Test olarak ayrılmıştır. 

training : 12753
validation : 3712
test : 7382

In [ ]:

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
train_dataset = get_training_dataset_preview(ordered=True)
y_train = next(iter(train_dataset.unbatch().map(lambda image, label: label).batch(NUM_TRAINING_IMAGES))).numpy()
print(f'Number of training images {NUM_TRAINING_IMAGES}')


NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
valid_dataset = get_validation_dataset(ordered=True)
y_valid = next(iter(valid_dataset.unbatch().map(lambda image, label: label).batch(NUM_VALIDATION_IMAGES))).numpy()
print(f'Number of validation images {NUM_VALIDATION_IMAGES}')


NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print(f'Number of test images {NUM_TEST_IMAGES}')
test_dataset = get_test_dataset(ordered=True)

In [ ]:
display_batch_of_images(next(iter(train_dataset.unbatch().batch(16))))
train_agg = np.asarray([[label, (y_train == index).sum()] for index, label in enumerate(CLASSES)])
valid_agg = np.asarray([[label, (y_valid == index).sum()] for index, label in enumerate(CLASSES)])






# Modeller

İlk Model oluşturuldu: Resnet, EfficientNetB6, EfficientNetB7, DenseNet102, DenseNet201 kullanılabilir.Deneyerek performansını görmek gerekir

"import efficientnet.tfkeras as efn" ile efficientnet hazır modeli keras'tan çağrılabilir.

* Son layer N_CLASSES (sınıf sayısı=104) kadar nöron içeren softmax olacaktır.
* Giriş 512,512,3
* Transfer learning bu case için çok faydalı olacaktır. noisy-student veya imagenet içerisinde çok fazla çiçek fotoğrafı olduğundan dolayı, pre-trained model kullanarak eğitime başlamak avantajlıdır. 

Pre-trained model kullanırken eğitim yaptığınız konu çok önemlidir, eğer herhangi bir pre-trained modelini içerisinde olmayan örnekler üzerine çalışıyorsanız pre-trained model avantaj yaratmayacaktır.

In [ ]:
def create_model(input_shape, N_CLASSES):
    base_model = efn.EfficientNetB6(weights='imagenet', 
                                    include_top=False,
                                    input_shape=input_shape)

    base_model.trainable = False # Freeze layers
    model = tf.keras.Sequential([
        base_model,
        L.GlobalAveragePooling2D(),
        L.Dense(N_CLASSES, activation='softmax')])
    
    return model

Model için Adam optimizer kullanılıldı. Burada farklı şeyler kullanmak mümkün, deneme yanılma ile seçmek gerekecek. Deneme yanılma ile seçtiğimiz parametrelere "hyper parametre" deriz. Bu parametreler Cross-validation dediğimiz çapraz gerçekleme ile belirlenmelidir.

loss için ise categorical corss entropy kullanılabileceği gibi, Sparse Categorical Crossentropy de kullanılabilir.
bknz: https://keras.io/api/optimizers/

In [ ]:
with strategy.scope():
    model = create_model((None, None, CHANNELS), N_CLASSES)
    
metric_list = ['sparse_categorical_accuracy']

optimizer = optimizers.Adam(lr=WARMUP_LEARNING_RATE)
model.compile(optimizer=optimizer, 
              loss=losses.SparseCategoricalCrossentropy(), 
              metrics=metric_list)
model.summary()

EfficientNetB6 modeli için ön ısınma turu "warm up" konulabilir. Burada 3 epoch layerları dondurulmuş modeli görmekteyiz, çok bir katkısı olmayabilir. Ama genelde büyük modeller üstünde transfer learning yaparken belirli bir epcoh sayısında ilk layerları dondurarak daha çok son layerları eğitiriz. Bunun sebebi ilk layerların temel geometrik şekilleri ve anlamları öğrenmesidir. 

In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
warmup_history = model.fit(x=get_training_dataset(), 
                           steps_per_epoch=STEPS_PER_EPOCH, 
                           validation_data=get_validation_dataset(),
                           epochs=WARMUP_EPOCHS, 
                           verbose=1).history


Learning rate'i dinamik vermek mümkündür. Burada farklı stratejiler güdülebilir. Ama temel trend düşük bir learning rate'den başlayarak önce biraz artırmak sonra azaltmaktır. Bu işin temel mantığı optimizasyon teorisine kadar gider. yapay sinir ağları hafıza bakımından en iyi gradient descent tarzı basit optimizasyon modelleri ile verimli çalıştığından dolayı, learning rate ana parametredir ( ilk başlangıç noktası da önemlidir, onu pre-trained model ile sağlıyoruz ). Local minimalara takılmadan gitmek için bu tarz stratejileri öneririm. Bu stratejiyi daha sonra model'i fit ederken callsback_learningrate olarak tanımlayacağız.

In [ ]:

LR_START = 0.00000001
LR_MIN = 0.000001
LR_MAX = LEARNING_RATE
LR_RAMPUP_EPOCHS = 4
LR_SUSTAIN_EPOCHS = 1
LR_EXP_DECAY = .81

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]

sns.set(style='whitegrid')
fig, ax = plt.subplots(figsize=(20, 7))
plt.plot(rng, y)

print(f'{EPOCHS} total epochs and {NUM_TRAINING_IMAGES//BATCH_SIZE} steps per epoch')
print(f'Learning rate schedule: {y[0]:.3g} to {max(y):.3g} to {y[-1]:.3g}')

Dondurduğumuz layerları açtık. Tüm modeli yazdık.

optimizer https://keras.io/api/optimizers/ 'dan seçilebilir. Nadam, Adam, SGD, RMSprop optimizerları arasından en iyi perform verecek optimizer cross-validation ile seçilebilir.

In [ ]:
for layer in model.layers:
    layer.trainable = True # Unfreeze layers

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', mode='min', save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, 
                   restore_best_weights=True, verbose=1)
lr_callback = LearningRateScheduler(lrfn, verbose=0)

callback_list = [checkpoint, es, lr_callback]

optimizer = optimizers.Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=metric_list)
model.summary()

SVG(tf.keras.utils.model_to_dot(model, dpi=70).create(prog='dot', format='svg'))

Modeli çalıştırdırk.

In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

history = model.fit(x=get_training_dataset(), 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_data=get_validation_dataset(), 
                    callbacks=callback_list, 
                    epochs=30, 
                    verbose=1).history

Loss grafiği

In [ ]:
def plot_metrics(history, metric_list):
    fig, axes = plt.subplots(len(metric_list), 1, sharex='col', figsize=(24, 12))
    axes = axes.flatten()
    
    for index, metric in enumerate(metric_list):
        axes[index].plot(history[metric], label=f'Train {metric}')
        axes[index].plot(history[f'val_{metric}'], label=f'Validation {metric}')
        axes[index].legend(loc='best', fontsize=16)
        axes[index].set_title(metric)

    plt.xlabel('Epochs', fontsize=16)
    sns.despine()
    plt.show()

plot_metrics(history, metric_list=['loss', 'sparse_categorical_accuracy'])

# Test Sonuçları

In [ ]:
x_test = test_dataset.map(lambda image, idnum: image)
test_preds = model.predict(x_test)
test_preds = np.argmax(test_preds, axis=-1)

test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

submission = pd.DataFrame(test_ids, columns=['id'])
submission['label'] = test_preds
submission.to_csv('submission.csv', index=False)
display(submission.head(12))

# DenseNet201 - Optional

DenseNet102 ve DenseNet201 de güzel modellerdir. Dense yapısı ile başarım göstermektedirler. Efficient Net çoğu durumda DenseNEt'i outperform etse dahi dataya göre farklı öodeller denemek gerekir.

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet201
import cv2
from IPython.display import SVG





def create_model2(input_shape, N_CLASSES):
    model2 = tf.keras.models.Sequential()

    model2.add(DenseNet201(
                     input_shape=(512,512,3),
                     weights = 'imagenet',
                     include_top=False))

    model2.add(tf.keras.layers.GlobalAveragePooling2D())

    model2.add(tf.keras.layers.Dense(N_CLASSES,
                                   activation='softmax'))


    
    return model2

with strategy.scope():
    model2 = create_model2((None, None, CHANNELS), N_CLASSES)
    


checkpoint = ModelCheckpoint(model_path, monitor='val_loss', mode='min', save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, 
                   restore_best_weights=True, verbose=1)
lr_callback = LearningRateScheduler(lrfn, verbose=0)

callback_list = [checkpoint, es, lr_callback]

optimizer = optimizers.Adam(lr=LEARNING_RATE)
model2.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=metric_list)
model2.summary()



    
SVG(tf.keras.utils.model_to_dot(model2, dpi=70).create(prog='dot', format='svg'))
                                   

In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
history = model2.fit(x=get_training_dataset(), 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_data=get_validation_dataset(), 
                    callbacks=callback_list, 
                    epochs=30, 
                    verbose=1).history

In [ ]:
print('Generating submission.csv file...')

# Get image ids from test set and convert to unicode
test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids,test_preds]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv